In [91]:
import pandas as pd
import re

In [109]:
CATEGORIES = ["action", "adventure", "animation", "biography", "comedy", "crime", "documentary", "drama", "family",
              "fantasy", "film_noir", "history", "horror", "music", "musical", "mystery", "romance", "sci_fi",
              "short", "sport", "superhero", "thriller", "war", "western"]

In [110]:
dataframes = []

for genre in CATEGORIES:
    df = pd.read_csv(f"./data/{genre}_crawled_data.csv")
    dataframes.append(df)

df = pd.concat(dataframes)
df.head()

,genre,title,poster_link,description,labels
0,action,House of the Dragon,https://m.media-amazon.com/images/M/MV5BZjBiOG...,An internal succession war within House Targar...,Action Adventure Drama
1,action,The Lord of the Rings: The Rings of Power,https://m.media-amazon.com/images/M/MV5BNTg3Nj...,Epic drama set thousands of years before the e...,Action Adventure Drama
2,action,Black Adam,https://m.media-amazon.com/images/M/MV5BYzZkOG...,"Nearly 5,000 years after he was bestowed with ...",Action Adventure Fantasy
3,action,Andor,https://m.media-amazon.com/images/M/MV5BNDgxNT...,Prequel series to Star Wars' 'Rogue One'. In a...,Action Adventure Drama
4,action,The School for Good and Evil,https://m.media-amazon.com/images/M/MV5BNzM1OD...,Best friends Sophie and Agatha find themselves...,Action Comedy Drama


In [111]:
len(df)

7200

### Convert labels string to one hot encoding

Add superhero label to labels (because IMDB guys to crazy)

In [112]:
labels = df.labels.values
genres = df.genre.values

df.labels = labels + " " + genres

Squash repeated films

In [113]:
df_agg = df.groupby("title").aggregate({"genre": 'first', "description": 'first', "labels": " ".join}).reset_index()
df_agg.head()

,title,genre,description,labels
0,'71,war,"In 1971, a young and disoriented British soldi...",Action Crime Drama war
1,'83,sport,"On June 25, 1983, the Lord's Cricket Ground wi...",Biography Drama History sport
2,'Allo 'Allo!,history,"In France during World War II, René Artois run...",Comedy History War history Comedy History War war
3,10 Cloverfield Lane,sci_fi,A young woman is held in an underground bunker...,Drama Horror Mystery sci_fi
4,10 Things I Hate About You,comedy,"A pretty, popular teenager can't go out on a d...",Comedy Drama Romance comedy Comedy Drama Roman...


Convert labels to one got encoding

In [114]:
set_of_all_labels = set()

for item in df.labels.apply(lambda x: list(x.split())):
    set_of_all_labels.update(item)

print(len(set_of_all_labels))
set_of_all_labels

51


{'Action',
 'Adventure',
 'Animation',
 'Biography',
 'Comedy',
 'Crime',
 'Documentary',
 'Drama',
 'Family',
 'Fantasy',
 'Film-Noir',
 'Game-Show',
 'History',
 'Horror',
 'Music',
 'Musical',
 'Mystery',
 'News',
 'Reality-TV',
 'Romance',
 'Sci-Fi',
 'Short',
 'Sport',
 'Talk-Show',
 'Thriller',
 'War',
 'Western',
 'action',
 'adventure',
 'animation',
 'biography',
 'comedy',
 'crime',
 'documentary',
 'drama',
 'family',
 'fantasy',
 'film_noir',
 'history',
 'horror',
 'music',
 'musical',
 'mystery',
 'romance',
 'sci_fi',
 'short',
 'sport',
 'superhero',
 'thriller',
 'war',
 'western'}

In [115]:
def convert_string_to_genre(string):
    return string.lower().replace("-", "_")

In [116]:
all_labels = [convert_string_to_genre(item) for item in set_of_all_labels]
all_labels.extend(df.genre.unique())
all_labels

['news',
 'drama',
 'comedy',
 'music',
 'romance',
 'comedy',
 'thriller',
 'music',
 'fantasy',
 'western',
 'film_noir',
 'thriller',
 'mystery',
 'documentary',
 'horror',
 'history',
 'family',
 'horror',
 'game_show',
 'family',
 'animation',
 'sci_fi',
 'adventure',
 'biography',
 'short',
 'short',
 'crime',
 'sci_fi',
 'sport',
 'reality_tv',
 'war',
 'action',
 'musical',
 'documentary',
 'western',
 'drama',
 'mystery',
 'musical',
 'war',
 'animation',
 'adventure',
 'fantasy',
 'romance',
 'crime',
 'biography',
 'history',
 'sport',
 'action',
 'superhero',
 'film_noir',
 'talk_show',
 'action',
 'adventure',
 'animation',
 'biography',
 'comedy',
 'crime',
 'documentary',
 'drama',
 'family',
 'fantasy',
 'film_noir',
 'history',
 'horror',
 'music',
 'musical',
 'mystery',
 'romance',
 'sci_fi',
 'short',
 'sport',
 'superhero',
 'thriller',
 'war',
 'western']

In [117]:
existed_labels = list(set(CATEGORIES).intersection(all_labels))
existed_labels.sort()
print(len(existed_labels))
existed_labels

24


['action',
 'adventure',
 'animation',
 'biography',
 'comedy',
 'crime',
 'documentary',
 'drama',
 'family',
 'fantasy',
 'film_noir',
 'history',
 'horror',
 'music',
 'musical',
 'mystery',
 'romance',
 'sci_fi',
 'short',
 'sport',
 'superhero',
 'thriller',
 'war',
 'western']

In [118]:
one_hots = []

for item in df_agg.labels:
    one_hot = [1 if genre in convert_string_to_genre(item) else 0 for genre in existed_labels]
    one_hots.append(one_hot)

In [119]:
df = df_agg

In [120]:
df[existed_labels] = one_hots

In [122]:
df.head()

,title,genre,description,labels,action,adventure,animation,biography,comedy,crime,...,musical,mystery,romance,sci_fi,short,sport,superhero,thriller,war,western
0,'71,war,"In 1971, a young and disoriented British soldi...",Action Crime Drama war,1,0,0,0,0,1,...,0,0,0,0,0,0,0,0,1,0
1,'83,sport,"On June 25, 1983, the Lord's Cricket Ground wi...",Biography Drama History sport,0,0,0,1,0,0,...,0,0,0,0,0,1,0,0,0,0
2,'Allo 'Allo!,history,"In France during World War II, René Artois run...",Comedy History War history Comedy History War war,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,1,0
3,10 Cloverfield Lane,sci_fi,A young woman is held in an underground bunker...,Drama Horror Mystery sci_fi,0,0,0,0,0,0,...,0,1,0,1,0,0,0,0,0,0
4,10 Things I Hate About You,comedy,"A pretty, popular teenager can't go out on a d...",Comedy Drama Romance comedy Comedy Drama Roman...,0,0,0,0,1,0,...,0,0,1,0,0,0,0,0,0,0


In [123]:
df.drop(columns=["labels"], inplace=True)

In [124]:
df.head()

,title,genre,description,action,adventure,animation,biography,comedy,crime,documentary,...,musical,mystery,romance,sci_fi,short,sport,superhero,thriller,war,western
0,'71,war,"In 1971, a young and disoriented British soldi...",1,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,1,0
1,'83,sport,"On June 25, 1983, the Lord's Cricket Ground wi...",0,0,0,1,0,0,0,...,0,0,0,0,0,1,0,0,0,0
2,'Allo 'Allo!,history,"In France during World War II, René Artois run...",0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,1,0
3,10 Cloverfield Lane,sci_fi,A young woman is held in an underground bunker...,0,0,0,0,0,0,0,...,0,1,0,1,0,0,0,0,0,0
4,10 Things I Hate About You,comedy,"A pretty, popular teenager can't go out on a d...",0,0,0,0,1,0,0,...,0,0,1,0,0,0,0,0,0,0


In [126]:
len(df)

4188

### Let's compute column with path to poster

In [127]:
def from_title_to_poster_path(genre, title):
    transformed_title = re.sub("\\W", "_", title.casefold()) + ".jpg"
    return "./data/" + genre + "/" + transformed_title

In [128]:
poster_pathes = []

for genre, title in list(zip(df["genre"], df["title"])):
    poster_pathes.append(from_title_to_poster_path(genre, title))

In [129]:
df["poster_path"] = poster_pathes

In [130]:
df.head()

,title,genre,description,action,adventure,animation,biography,comedy,crime,documentary,...,mystery,romance,sci_fi,short,sport,superhero,thriller,war,western,poster_path
0,'71,war,"In 1971, a young and disoriented British soldi...",1,0,0,0,0,1,0,...,0,0,0,0,0,0,0,1,0,./data/war/_71.jpg
1,'83,sport,"On June 25, 1983, the Lord's Cricket Ground wi...",0,0,0,1,0,0,0,...,0,0,0,0,1,0,0,0,0,./data/sport/_83.jpg
2,'Allo 'Allo!,history,"In France during World War II, René Artois run...",0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,1,0,./data/history/_allo__allo_.jpg
3,10 Cloverfield Lane,sci_fi,A young woman is held in an underground bunker...,0,0,0,0,0,0,0,...,1,0,1,0,0,0,0,0,0,./data/sci_fi/10_cloverfield_lane.jpg
4,10 Things I Hate About You,comedy,"A pretty, popular teenager can't go out on a d...",0,0,0,0,1,0,0,...,0,1,0,0,0,0,0,0,0,./data/comedy/10_things_i_hate_about_you.jpg


In [132]:
df.drop(columns=["genre"], inplace=True)

In [133]:
df.to_csv("./data/preprocessed.csv", index=False)